In [ ]:
import os
import pickle
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as offline
import plotly.graph_objs as go
from Cluster_analysis import *


In [ ]:
PATH = 'Data/final_df.csv'
# Load final_df from csv file
df = pd.read_csv(PATH, sep='\t')
# Fill missing values with 'Not Available' for all columns except 'Partner'
df = df.fillna('Not Available')
# Replace 'Not Available' with 'No partner' for 'Partner' column
df['partner'] = df['partner'].replace('Not Available', 'No partner')
df.head(3)


## Full graph

In [ ]:
#computes centroid of each cluster
def compute_centroids(df, column):
    centroids = []
    for i in range(df[column].max()+1):
        centroids.append(df[df[column] == i][['X', 'Y', 'Z']].mean().values)
    return centroids

centroids = compute_centroids(df, 'labels')

# Plotly 3D scatter plot
fig = px.scatter_3d(df, x='X', y='Y', z='Z', color='labels', 
                    color_continuous_scale=px.colors.cyclical.HSV, hover_name='Character name', hover_data={'Name': True,
                    'Release date': True, 'Actor name': True, 'Actor age at release': True, 'Gender': True, 'X': False,
                    'Y': False, 'Z': False, 'labels': False, 'title': True, 'filtered_descriptions': True, 'partner': True})

fig.update_traces(marker=dict(size=3, colorscale='Viridis'), marker_line_width=1, marker_line_color='DarkSlateGrey', hovertemplate =
            "<b>%{hovertext}</b><br><br>" +
            "Movie: %{customdata[0]}<br>" +
            "Release date: %{customdata[1]}<br>" +
            "Actor name: %{customdata[2]}<br>" +
            "Actor age at release: %{customdata[3]}<br>" +
            "Gender: %{customdata[4]}<br>" +
            "Character archetype: %{customdata[6]}<br>"),
            #"Description: %{customdata[8]}<br>"),

fig = add_centroids(fig, centroids)

fig = set_layout(fig)

# Make initial plot more zoomed in
fig.update_layout(scene_camera_eye=dict(x=0.5, y=0.5, z=0.5))

# Make the height of the plot larger
fig.update_layout(height=800)

fig.show()
fig.write_html("Plot4Website.html")